In [1]:
%config Completer.use_jedi = False
%load_ext autoreload

# Imports 

In [2]:
from crawler.src import recommender

In [132]:
import pandas as pd
from scipy.sparse import coo_matrix, csr_matrix
import numpy as np
from sklearn.preprocessing import LabelEncoder
from implicit.als import AlternatingLeastSquares
from scipy import sparse
import csv
from time import time

In [3]:
pd.options.display.max_columns=200
pd.options.display.max_rows=300

In [5]:
df_rank = pd.read_csv("../../data/crawler/boardgame_ranks_20250526.csv")
df_rank.head()

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank,queried_at_utc
0,224517,Brass: Birmingham,2018,1,8.40146,8.57715,52086,0,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,2025-05-26T10:36:34
1,161936,Pandemic Legacy: Season 1,2015,2,8.36386,8.51619,55588,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,2025-05-26T10:36:34
2,342942,Ark Nova,2021,3,8.34620,8.53657,52459,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,2025-05-26T10:36:34
3,174430,Gloomhaven,2017,4,8.32586,8.56341,64874,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,2025-05-26T10:36:34
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.22914,8.57682,26087,0,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,2025-05-26T10:36:34


In [45]:
df_rank.shape

(165498, 17)

In [46]:
df_rank.loc[df_rank["is_expansion"]==0].shape

(129576, 17)

# Load parquet file

In [43]:
df_bg_data_raw = pd.read_parquet("../../data/boardgame_data/boardgame_data_raw_1748311285.parquet")
print(df_bg_data_raw.shape)
df_bg_data_raw.head()

(42486, 36)


,game_id,thumbnail,image,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions
0,13,https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI...,https://cf.geekdo-images.com/PyUol9QxBnZQCJqZI...,3,4,120,60,120,10,"[Economic, Negotiation]","[Chaining, Dice Rolling, Hexagon Grid, Income,...","[Animals: Sheep, Components: Hexagonal Tiles, ...","[20 Jahre Darmstadt Spielt, Brettspiel Advents...","[Volkan Baga, Tanja Donner, Pete Fenlon, Jason...","[CATAN 3D Collector's Edition, Catan: 25 Jahre...","[Baden-Württemberg Catan, Catan Geographies: G...",[Klaus Teuber],"[KOSMOS, 64 Ounce Games, 999 Games, Albi, Asmo...",[],1.50101,0.0,2.2871,223336,2306,519,7567,22919,8360,135593,"{'total_votes': 577, '2': 1, '3': 0, '4': 2, '...","{'50 percent': '8', '25 percent': '8', '75 per...","{'total_votes': 346, '1': 21, '2': 294, '3': 3...",{'75 percent': 'Some necessary text - easily m...,"{'Not Recommended': ['1', '2', '4+'], 'Recomme...","{'total_votes': 2071, '1': {'Best': 1, 'Recomm...","[{'version_id': 467634, 'version_nickname': 'A..."
1,822,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,https://cf.geekdo-images.com/okM0dq_bEXnbyQTOv...,2,5,45,30,45,7,"[Medieval, Territory Building]","[Area Majority / Influence, Enclosure, End Gam...","[Category: Dized Tutorial, Cities: Carcassonne...","[20 Jahre Darmstadt Spielt, Antwerpen (fan exp...","[Marcel Gröber, Doris Matthäus, Anne Pätzke, C...","[Carcassonne Big Box, Carcassonne Big Box 2, C...","[The Ark of the Covenant, Carcassonne Demo, Ca...",[Klaus-Jürgen Wrede],"[Hans im Glück, 64 Ounce Games, 999 Games, Alb...","[Carcassonne: Wheel of Fortune, Mists over Car...",1.31104,0.0,1.8880,209535,2034,689,10188,22592,8466,134541,"{'total_votes': 766, '2': 2, '3': 1, '4': 12, ...","{'25 percent': '6', '75 percent': '8', '50 per...","{'total_votes': 490, '6': 481, '7': 7, '8': 1,...","{'75 percent': 'No necessary in-game text', '5...","{'Not Recommended': ['1', '5+'], 'Best': ['2',...","{'total_votes': 2659, '1': {'Best': 7, 'Recomm...","[{'version_id': 31146, 'version_nickname': 'Al..."
2,30549,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,https://cf.geekdo-images.com/S3ybV1LAp-8SnHIXL...,2,4,45,45,45,8,"[Medical, Travel]","[Action Points, Chaining, Contracts, Cooperati...","[Components: Map (Global Scale), Components: M...",[Pandemic: Gen Con 2016 Promos – Z-Force Team ...,"[Josh Cappel, Christian Hanisch, Régis Moulun,...",[],"[Fall of Rome, Iberia, Pandemic Legacy: Season...",[Matt Leacock],"[Z-Man Games, (Unknown), ADC Blackfire Enterta...",[],1.33738,0.0,2.3968,214983,3251,619,11200,20067,6189,131235,"{'total_votes': 608, '2': 0, '3': 0, '4': 1, '...","{'25 percent': '8', '75 percent': '10', '50 pe...","{'total_votes': 341, '11': 10, '12': 279, '13'...",{'75 percent': 'Some necessary text - easily m...,"{'Recommended': ['1', '2', '3'], 'Best': ['4']...","{'total_votes': 2123, '1': {'Best': 170, 'Reco...","[{'version_id': 30851, 'version_nickname': 'Al..."
3,68448,https://cf.geekdo-images.com/35h9Za_JvMMMtx_92...,https://cf.geekdo-images.com/35h9Za_JvMMMtx_92...,2,7,30,30,30,10,"[Ancient, Card Game, City Building, Civilizati...","[Closed Drafting, End Game Bonuses, Hand Manag...","[Ancient: Babylon, Ancient: Egypt, Ancient: Gr...","[7 Wonders: Armada, 7 Wonders: Armada – Siracu...","[Dimitri Chappuis, Miguel Coimbra, Etienne Heb...",[],"[7 Wonders (Second Edition), 7 Wonders Duel, 7...",[Antoine Bauza],"[Repos Production, ADC Blackfire Entertainment...",[],1.27709,0.0,2.3158,149251,1934,985,14419,16874,5414,109297,"{'total_votes': 561, '2': 0, '3': 0, '4': 1, '..."

In [44]:
df_bg_data_raw.tail()

,game_id,thumbnail,image,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions
42481,39412,https://cf.geekdo-images.com/ClGWqwZDx5SJgOjh2...,https://cf.geekdo-images.com/ClGWqwZDx5SJgOjh2...,2,10,0,0,0,0,"[Expansion for Base-game, Fighting, Humor, Mat...","[Critical Hits and Failures, Dice Rolling, Gri...","[Creatures: Aliens / Extraterrestrials, Theme:...",[Macho Women with Guns: The Final Chapter (Par...,[Richard Menustik],[],[],[Greg Porter],[Blacksburg Tactical Research Center],"[Macho Women with Guns, Renegade Nuns on Wheels]",0.00000,0.0,0.0000,27,2,0,4,3,0,1,{},{},{},{},"{'Best': ['1', '2', '3', '4', '5', '6', '7', '...","{'total_votes': 0, '1': {'Best': 0, 'Recommend...",None
42482,39415,https://cf.geekdo-images.com/aXSB_LM1jYoA3a0l4...,https://cf.geekdo-images.com/aXSB_LM1jYoA3a0l4...,2,6,60,60,60,12,"[Electronic, Party Game, Trivia]",[Roll / Spin and Move],[Game: Trivial Pursuit],[],[],[],[],[(Uncredited)],"[Parker Brothers, Electronic Arts Inc. (EA), H...",[],1.84127,0.0,1.3333,488,25,1,6,46,3,142,{},{},"{'total_votes': 1, '41': 0, '42': 0, '43': 0, ...",{'75 percent': 'Unplayable in another language...,"{'Not Recommended': ['1'], 'Recommended': ['2'...","{'total_votes': 1, '1': {'Best': 0, 'Recommend...","[{'version_id': 32942, 'width': 11, 'length': ..."
42483,39417,https://cf.geekdo-images.com/k9jxqwpTuW_zTgeVl...,https://cf.geekdo-images.com/k9jxqwpTuW_zTgeVl...,2,4,0,0,0,3,"[Children's Game, Fantasy]",[Roll / Spin and Move],"[Authors: Charles Perrault, Authors: The Broth...",[],[],[],[],[(Uncredited)],"[University Games, University Games]",[],0.00000,0.0,0.0000,0,0,0,0,0,0,0,{},{},{},{},"{'Best': ['1', '2', '3', '4', '4+']}","{'total_votes': 0, '1': {'Best': 0, 'Recommend...",None
42484,39423,https://cf.geekdo-images.com/IIyLKkMcl0Vw0z0C4...,https://cf.geekdo-images.com/IIyLKkMcl0Vw0z0C4...,1,3,5,5,5,0,[Children's Game],[],[],[],[],[],[],[(Uncredited)],[Användbart Litet Företag],[],1.69967,0.0,1.0000,6,0,0,0,1,1,3,"{'total_votes': 1, '2': 1, '3': 0, '4': 0, '5'...","{'75 percent': '2', '50 percent': '2', '25 per...",{},{},"{'Best': ['1', '2', '3', '3+']}","{'total_votes': 0, '1': {'Best': 0, 'Recommend...",None
42485,39424,https://cf.geekdo-images.com/PTh678EsZZ7Gofl98...,https://cf.geekdo-images.com/PTh678EsZZ7Gofl98...,2,8,0,0,0,0,"[Action / Dexterity, Sports]",[],[Sports: Golf],[],[],[],[],[(Uncredited)],"[Parker Brothers, Parker Brothers]",[],0.00000,0.0,0.0000,1,0,0,0,0,0,0,{},{},{},{},"{'Best': ['1', '2', '3', '4', '5', '6', '7', '...","{'total_votes': 0, '1': {'Best': 0, 'Recommend...",None


In [ ]:
df_bg_data_raw

In [28]:
df_bg_data_raw.loc[df_bg_data_raw["game_id"]==2627]

,game_id,thumbnail,image,minplayers,maxplayers,playingtime,minplaytime,maxplaytime,minage,boardgamecategory,boardgamemechanic,boardgamefamily,boardgameexpansion,boardgameartist,boardgamecompilation,boardgameimplementation,boardgamedesigner,boardgamepublisher,boardgameintegration,stddev,median,averageweight,owned,trading,wanting,wishing,numcomments,numweights,numratings,suggested_playerage,suggested_playerage_quartiles,language_dependence,language_dependence_quartiles,player_count_recs,suggested_numplayers,versions


In [52]:
df_ratings = pd.read_parquet("../../data/boardgame_data/boardgame_rating_data_1748319041.parquet")
print(df_ratings.shape)
df_ratings.head()

(1420, 20)


,game_id,2.0,3.0,4.0,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,1.0,2.5,4.5,3.5,1.5
0,179350,"[obifrankenobi, cfarrell]","[Carlos Palomino, Beorne, asiok, Jetseb, Janon...","[Grildensnork, Murdockus, vetinari7878, Rabbit...","[Nico D, woj_settlers, halbert82, Chuckila, di...","[Bryo4, carontissimo, LookAtTheBacon, sillue, ...","[nunovix, Wendigohexe, zyxyv, hedisch, tgbruis...","[Orphebus, Kuula, littlebanshee, monkeykenobi,...","[MIKI1980, dbarbaglia, teamat, PlayThisToday, ...","[s-man, Mottchen, dpkmc, Surya, Pum_, LucaLeon...","[stoneghe, KABIA66, skipper2206, Turbo2000, ka...","[nunoR, liga, Quintious, manucube, avanspronse...","[Faustoxx, the_spy, Olaf78, Racing cards, dtiv...","[nintakin, nikalibi]","[Gio8, BhART, franck1912, jrubin1, jerome53240...",None,None,None,None,None
1,99808,"[gimmster, saxophonechapa, eduart, DavidMcGreg...","[Randy Cox, hkhutchi, Gyges, Kaelistus, garg, ...","[Jbrady, mmtmr, sfox, Cerebremancer, cdefrisco...","[dhailey, Grildensnork, Justinschaffer, jgoyes...","[Lothartvni, BC Indy, nittany0508, fehrmeister...","[HooeyPooey, kiddynamite, zefquaavius, Bobby4t...","[TundraThunder, JensFL, Menma, Guiannos, Mibao...","[Errante, maydaygames, kingofthegrill, mechani...","[d_hoopster, takras, fringer, jeremycoe, Conne...","[DAZHEL, MikoML, Boots01, Braidrock, nymartin2...","[mtn55, soulyogurt, eiopqrtuwy, theapostlegree...","[kmbx13, b4kabob, prosfilaes, NerdDad, kommand...",None,"[Flambers, CRRXVI, paintime, TheBardGamer]","[paulo, McCrank, Kenig]",[kcayoung],"[Spyzou, Mario Pawlowski, Zepheus, Janon, smil...",None,None
2,286667,[idontplayco_ops],"[dream spawn, aranvulnep, Janasik, realwizard,...","[raile, jonpurkis, AmandaDesignsGames, StellaH...","[povertidonki, Invicta Bricks, ryansmum2008, s...","[stroutqb22, Elpete, Duke_Fleed, rsj77, in8nir...","[feldfan2014, cabbagekingaf, Innsmouth_Look, j...","[boardingschoolbrandi, acecondor, drulive99, h...","[QuixoticQuixotic, fencedingates, AerynSpaceFa...","[TaijiJohn, Samutom84, ctrosejr, Jim1701, iuga...","[AHforever, ku8843, Wolverutto, warta, JimmyEW...","[Grenade, FrostyMeeple, sorcretic, Renno]","[RichardIngram, cardshark28800, jcadillac, cft...",None,"[matty0226, Little_Rocket_Games, alessandra198...","[ultimoratio, hazered, dakotamassie]",None,[radiofyr309],None,None
3,170041,"[scps14, Simcsa, danchi, cadhlan]","[imoo, missiloon, Sweetdogpaw, djhill, JohansG...","[StefanvM1, Livefree, leffe dubbel, Drval, woj...","[Acido, crazyclark, takokawa, river911, costel...","[Menma, Femtitala, linkme, pethulhu, Dalkuldar...","[morishita, Hanno, SplitCreek, Molotail, Great...","[rjgomes, MattCNixon, cogo71, Izobretenik, pec...","[pecotti, DeDaan, nivatra, Frontliner, Els76, ...","[tosh_loco, strubs42, gruntsudens, Bresto, epi...","[Gorkx77, deezynah, Wickedheart, s0915579408, ...","[Poeppelhoppers, vikachoo, MariBor]","[NatkaGra, pilgrim152, Tolkana, castor1966, Su...","[Rosiria, Marysinka]","[Noni1702, Racing cards, silenthoch, ThunderWo...",None,None,[Shamragnar],None,None
4,300993,[MaxRokotansky],"[MrIkerito, Geeken, Giulii, chumy, elchabonque...","[Alesbolmol, RafaGR, ckomiya, josecarloscom, a...","[Erreka, RubenMS, Santycapi, Yorkaeff, maxiscp...","[nick_arg, MissRiuku, fustero, Norg222, Hoja]","[Juan_Aguilera, thisisrhun, Braculaventura, Ev...","[AngPredator, Ninorro, darthway, Jesseaglerock...","[HANJEL, MrCaronte77, pablo24091980, sandraast...","[Misiga, facalrivas, Guanarteme80, Parsifal1, ...","[peterpax, Jowyjokes, Alex_Luka23, Charly81, E...","[bichejote, Acoidan85, herbius, Whirlpool, mru...","[jdhorux, xelerad, PabloJimenezRoman, Zelgadis...","[iwancho, robinson_2017]","[NinyaSiniestro, Cascabel93, David S Serrano, ...",None,None,"[moncsal213, RedFate]",None,None


In [53]:
df_ratings.tail()

,game_id,2.0,3.0,4.0,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,1.0,2.5,4.5,3.5,1.5
1415,197439,None,None,"[apocatastasi, ratbulogg, hcbftw, Bangla, whit...","[King Gary, Tikigod, KRWoods1, Peepser, Preach...","[Ephastion, StephenV, REGGY, djboss, Leo-T]","[prd1982, cfarrell, spadovsky, Michael Ward, t...","[Murdockus, Svarthand, marge2, ryanabrams, dki...","[jamomoto, frankstarro, stoneghe, e.c.r., Benj...","[slashing, LucaLeoncavallo, chg21012, BerenBar...","[Geeken, roro77, bluekingzog, hockeyking9, jan...","[jahco1980, Tuttle757, devaed, Friedrich, itha...","[PierreP, echallstrom, Snoochies, Fogtmann, sa...",[aslskfan],"[drillSGT, wtdickens, Blake the Flake, idjohns...",[Zete],None,None,None,None
1416,424981,None,None,"[himokl, gepesso, pvtparts, azeem]","[arron77, slowPigeon, Bigshowfan, Oulibanoum]",None,"[fourquartets, evil_lemons, timtims1, jmdsplot...","[Victorshade, Earl of Dukes, dragos_br]","[Neale2006, Primertt, larryjrice, truthwritten...","[Escapist_Artist, Der Gwiedo, johnnysavs, g6gr...","[MaitreYo06, Moe33, feldfan2014, BobShick, Clo...","[bionicle4365, Darla, C_Cheung, joshbalvin, sp...","[trimall, billybob0101011, oatmeal1201, marsti...","[LordNoumenon, survivorfan, Japara, zenxacred,...","[Sketcherrrr, logopolys, MelbaMo, nfirehazzard...",None,None,None,None,None
1417,352591,None,None,"[lks524, Zylofan, bubblepipemedia, CMagex, him...","[kyk10pes, mdiep, docpooh18, Catyrpelius, self...","[webstergroup, minakobrown, JoshDip]","[GizmoBoardGames, Kayma, Primori, Darrius Tuck...","[crocko33, Mkyrmse, Cuelannoo, AegisToast, Aeg...","[boardgameclinic, Rafiki_Robert, xerossilence,...","[Andorra9200, Squalidsafe, Sir Melancholie, ta...","[challahback252, FreshBao, cardboardeaux, Jess...","[wobbly_chairs, PeteWiFi]","[steph_noack, Master Thomas, ACGalaga, Desertm...","[jeremiahbrammer, Kenra]","[jamboldr, Grey_Pilgrim, seaskytowers, Scott99...",None,None,[vyvyannexp],None,None
1418,181289,None,None,None,"[Tudada, xxlmc, FrAnkX0r88, BlueLycan, ChunmaV...","[Panta_rhei, chama_81]","[failtech, Joe2016, sykano, iquito, simon055, ...",[waitmind],"[boardmathias, jgbaxter, kaddimaus, zeronemo, ...","[knotsao, Mr. Panci, High Elf Andrew, gthanato...","[damboshi, zdsarq32, netbus22, virginis, bluem...","[Spielkind78, hodoliiiii, Markus Shepherd, Hal...","[Mister_Peon, just_lis, nyravv, nimiralism, xp...","[Eurodancer, PMatheus, Ollip, fmmhide, Vermout...","[bignike_ch, jinlv81, Michiel, MarkuZa, irene0...","[justletmedownload22, Jimmy_Dean]",None,None,None,None
1419,351735,None,None,None,"[Spock71, apuity, barba, funktor, ryansdavis]",None,"[Shinn_S, wilsonza, Labbe, B2TB, Linkovich, hh...","[NICONIGLIO, lumiotso, Teurlinx, Htivante23, i...","[tribefan07, CassyCZ, fanta1995, firyfly, Coli...","[cinocinosnellman, 2goofy, Fumier, Imrhalis, P...","[danielklinge, HariSeldon86, vlcaczech, eligam...","[norberts, xerxes2k, Suspectus, Parkettwischer...","[Swoosl, Buddha007, KatsuIas, Big Fat Penguin,...","[Ema_C, loopszz]","[misskitten, heliossone, NemesisWhip, LotusTha...",[heinhunstig],None,[k4k7uz],None,None


In [19]:
df_bg_data_raw["game_id"].nunique()

11768

# Recommender testing

In [50]:
df_rank.loc[df_rank["id"].isin(df_ratings["game_id"].astype(int).tolist())].sort_index().head(50)

,id,name,yearpublished,rank,bayesaverage,average,usersrated,is_expansion,abstracts_rank,cgs_rank,childrensgames_rank,familygames_rank,partygames_rank,strategygames_rank,thematic_rank,wargames_rank,queried_at_utc
1,161936,Pandemic Legacy: Season 1,2015,2,8.36386,8.51619,55588,0,NaN,NaN,NaN,NaN,NaN,2.0,1.0,NaN,2025-05-26T10:36:34
2,342942,Ark Nova,2021,3,8.34620,8.53657,52459,0,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,2025-05-26T10:36:34
3,174430,Gloomhaven,2017,4,8.32586,8.56341,64874,0,NaN,NaN,NaN,NaN,NaN,4.0,2.0,NaN,2025-05-26T10:36:34
4,233078,Twilight Imperium: Fourth Edition,2017,5,8.22914,8.57682,26087,0,NaN,NaN,NaN,NaN,NaN,5.0,4.0,NaN,2025-05-26T10:36:34
6,167791,Terraforming Mars,2016,7,8.20027,8.34848,106321,0,NaN,NaN,NaN,NaN,NaN,8.0,NaN,NaN,2025-05-26T10:36:34
9,162886,Spirit Island,2017,10,8.14215,8.34133,57486,0,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,2025-05-26T10:36:34
10,291457,Gloomhaven: Jaws of the Lion,2020,11,8.12757,8.39387,37838,0,NaN,NaN,NaN,NaN,NaN,11.0,7.0,NaN,2025-05-26T10:36:34
13,12333,Twilight Struggle,2005,14,8.05311,8.23676,50487,0,NaN,NaN,NaN,NaN,NaN,14.0,NaN,2.0,2025-05-26T10:36:34
14,182028,Through the Ages: A New Story of Civilization,2015,15,8.03713,8.26768,33348,0,NaN,NaN,NaN,NaN,NaN,13.0,NaN,NaN,2025-05-26T10:36:34
15,84876,The Castles of Burgundy,2011,16,8.02123,8.14946,65940,0,NaN,NaN,NaN,NaN,NaN,16.0,NaN,NaN,2025-05-26T10:36:34


In [30]:
df_ratings = pd.read_parquet("../../data/crawler/boardgame_ratings_1748670263.parquet")
print(df_ratings.shape)

(7459, 22)


In [55]:
import importlib
importlib.reload(recommender)

<module 'crawler.src.recommender' from '/home/ubuntu/git/pax_tt_recommender/crawler/src/recommender.py'>

In [56]:
recs = recommender.GameRecommender(min_ratings_per_user=5)
recs.fit(df_ratings.set_index("game_id"))

In [76]:
game_recs = recs.recommend_similar_games(
    game_ids=[
        "337627",
        "342942",
        "84876",
        "167791",
        "358661",
        "325494",
        "356754",
        "367966",
        "363622",
        "391163",
        "416851",
        "379629",
        "357563",
        "414317",
        "364073",
        "367220", 
        "393325"
    ],
    disliked_games=["124361", "360692", "162886", "242705", "387378", "346703"],
    n_recommendations=20,
)
df_rank.loc[df_rank["id"].isin([int(x[0]) for x in game_recs]), ["name", "id"]]

,name,id
52,Cascadia,295947
57,SETI: Search for Extraterrestrial Intelligence,418059
143,Wyrmspan,410201
200,Welcome to the Moon,339789
376,The Guild of Merchant Explorers,350933
413,The Vale of Eternity,385529
491,Let's Go! To Japan,368173
520,World Wonders,365258
555,Bomb Busters,413246
587,Trio,352515


In [77]:
game_recs

[('385529', np.float64(0.06785633913924893)),
 ('411875', np.float64(0.06053934934987508)),
 ('418059', np.float64(0.05840121686665953)),
 ('420087', np.float64(0.05533289248011547)),
 ('352515', np.float64(0.052347185592146184)),
 ('368173', np.float64(0.05031468468552838)),
 ('350933', np.float64(0.050026411196026854)),
 ('365258', np.float64(0.04820333995296913)),
 ('353545', np.float64(0.047814332285280786)),
 ('413246', np.float64(0.047633915639819085)),
 ('338093', np.float64(0.04447478084648265)),
 ('339789', np.float64(0.04306313781506729)),
 ('397385', np.float64(0.042361647799737945)),
 ('406854', np.float64(0.04184043808879839)),
 ('402676', np.float64(0.041562102540840584)),
 ('403754', np.float64(0.04133019349959283)),
 ('399941', np.float64(0.04111172453914003)),
 ('410201', np.float64(0.04101625461555941)),
 ('295947', np.float64(0.04097151138347144)),
 ('384213', np.float64(0.04074862547557892))]

6      Terraforming Mars
9          Spirit Island
32              Wingspan
36               Barrage
52              Cascadia
117             Carnegie
Name: name, dtype: object

In [ ]:
# Initialize and fit the recommender
recs = GameRecommender(min_ratings_per_user=3)
recs.fit(your_dataframe)

# Get recommendations for a user
user_recommendations = recs.recommend(user_id="user123", n_recommendations=5)

# OR get similar games based on a list of games
game_recommendations = recs.recommend(game_ids=["game1", "game2"], n_recommendations=5)

In [ ]:
GameRecommender

In [5]:
df_ratings.head()

,game_id,2.0,3.0,4.0,5.0,5.5,6.0,6.5,7.0,7.5,8.0,8.5,9.0,9.5,10.0,1.0,2.5,4.5,3.5,1.5,0.0,0.5
0,179350,"[obifrankenobi, cfarrell]","[Carlos Palomino, Beorne, asiok, Jetseb, Janon...","[Grildensnork, Murdockus, vetinari7878, Rabbit...","[Nico D, woj_settlers, halbert82, Chuckila, di...","[Bryo4, carontissimo, LookAtTheBacon, sillue, ...","[nunovix, Wendigohexe, zyxyv, hedisch, tgbruis...","[Orphebus, Kuula, littlebanshee, monkeykenobi,...","[MIKI1980, dbarbaglia, teamat, PlayThisToday, ...","[s-man, Mottchen, dpkmc, Surya, Pum_, LucaLeon...","[stoneghe, KABIA66, skipper2206, Turbo2000, ka...","[nunoR, liga, Quintious, manucube, avanspronse...","[Faustoxx, the_spy, Olaf78, Racing cards, dtiv...","[nintakin, nikalibi]","[Gio8, BhART, franck1912, jrubin1, jerome53240...",None,None,None,None,None,None,None
1,99808,"[gimmster, saxophonechapa, eduart, DavidMcGreg...","[Randy Cox, hkhutchi, Gyges, Kaelistus, garg, ...","[Jbrady, mmtmr, sfox, Cerebremancer, cdefrisco...","[dhailey, Grildensnork, Justinschaffer, jgoyes...","[Lothartvni, BC Indy, nittany0508, fehrmeister...","[HooeyPooey, kiddynamite, zefquaavius, Bobby4t...","[TundraThunder, JensFL, Menma, Guiannos, Mibao...","[Errante, maydaygames, kingofthegrill, mechani...","[d_hoopster, takras, fringer, jeremycoe, Conne...","[DAZHEL, MikoML, Boots01, Braidrock, nymartin2...","[mtn55, soulyogurt, eiopqrtuwy, theapostlegree...","[kmbx13, b4kabob, prosfilaes, NerdDad, kommand...",None,"[Flambers, CRRXVI, paintime, TheBardGamer]","[paulo, McCrank, Kenig]",[kcayoung],"[Spyzou, Mario Pawlowski, Zepheus, Janon, smil...",None,None,None,None
2,286667,[idontplayco_ops],"[dream spawn, aranvulnep, Janasik, realwizard,...","[raile, jonpurkis, AmandaDesignsGames, StellaH...","[povertidonki, Invicta Bricks, ryansmum2008, s...","[stroutqb22, Elpete, Duke_Fleed, rsj77, in8nir...","[feldfan2014, cabbagekingaf, Innsmouth_Look, j...","[boardingschoolbrandi, acecondor, drulive99, h...","[QuixoticQuixotic, fencedingates, AerynSpaceFa...","[TaijiJohn, Samutom84, ctrosejr, Jim1701, iuga...","[AHforever, ku8843, Wolverutto, warta, JimmyEW...","[Grenade, FrostyMeeple, sorcretic, Renno]","[RichardIngram, cardshark28800, jcadillac, cft...",None,"[matty0226, Little_Rocket_Games, alessandra198...","[ultimoratio, hazered, dakotamassie]",None,[radiofyr309],None,None,None,None
3,170041,"[scps14, Simcsa, danchi, cadhlan]","[imoo, missiloon, Sweetdogpaw, djhill, JohansG...","[StefanvM1, Livefree, leffe dubbel, Drval, woj...","[Acido, crazyclark, takokawa, river911, costel...","[Menma, Femtitala, linkme, pethulhu, Dalkuldar...","[morishita, Hanno, SplitCreek, Molotail, Great...","[rjgomes, MattCNixon, cogo71, Izobretenik, pec...","[pecotti, DeDaan, nivatra, Frontliner, Els76, ...","[tosh_loco, strubs42, gruntsudens, Bresto, epi...","[Gorkx77, deezynah, Wickedheart, s0915579408, ...","[Poeppelhoppers, vikachoo, MariBor]","[NatkaGra, pilgrim152, Tolkana, castor1966, Su...","[Rosiria, Marysinka]","[Noni1702, Racing cards, silenthoch, ThunderWo...",None,None,[Shamragnar],None,None,None,None
4,300993,[MaxRokotansky],"[MrIkerito, Geeken, Giulii, chumy, elchabonque...","[Alesbolmol, RafaGR, ckomiya, josecarloscom, a...","[Erreka, RubenMS, Santycapi, Yorkaeff, maxiscp...","[nick_arg, MissRiuku, fustero, Norg222, Hoja]","[Juan_Aguilera, thisisrhun, Braculaventura, Ev...","[AngPredator, Ninorro, darthway, Jesseaglerock...","[HANJEL, MrCaronte77, pablo24091980, sandraast...","[Misiga, facalrivas, Guanarteme80, Parsifal1, ...","[peterpax, Jowyjokes, Alex_Luka23, Charly81, E...","[bichejote, Acoidan85, herbius, Whirlpool, mru...","[jdhorux, xelerad, PabloJimenezRoman, Zelgadis...","[iwancho, robinson_2017]","[NinyaSiniestro, Cascabel93, David S Serrano, ...",None,None,"[moncsal213, RedFate]",None,None,None,None


In [ ]:
# Step 1: Flatten and explode data
rows = []
for _, row in df_ratings.iterrows():
    for rating in df_ratings.columns:
        users = row[rating]
        game_id = row["game_id"]
        if isinstance(users, list):
            for user in users:
                rows.append((user, game_id, float(rating)))
# Convert to DataFrame
ratings_df = pd.DataFrame(rows, columns=['user', 'game', 'rating'])   

# Count ratings per user
user_counts = ratings_df['user'].value_counts()

# Filter users with at least 3 ratings
valid_users = user_counts[user_counts >= 3].index

# Filter the dataframe
filtered_ratings_df = ratings_df[ratings_df['user'].isin(valid_users)].copy()

# Step 2: Encode users and games as integer IDs
user_encoder = LabelEncoder()
game_encoder = LabelEncoder()

filtered_ratings_df['user_id'] = user_encoder.fit_transform(filtered_ratings_df['user'])
filtered_ratings_df['game_id'] = game_encoder.fit_transform(filtered_ratings_df['game'])

# Step 3: Create sparse matrix
user_ids = filtered_ratings_df['user_id'].values
game_ids = filtered_ratings_df['game_id'].values
ratings = filtered_ratings_df['rating'].values

ratings_sparse_matrix = coo_matrix((ratings, (user_ids, game_ids)))
print(ratings_sparse_matrix.shape)
# del(filtered_ratings_df)
# del(ratings_df)
# del(rows)

In [ ]:
item_user_matrix = csr_matrix(ratings_sparse_matrix.T)

model = AlternatingLeastSquares(factors=50, regularization=0.1, iterations=20)
model.fit(item_user_matrix)

# Recommender script testing

In [2]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from typing import Dict, List, Tuple
import argparse
import logging
from pathlib import Path
import json

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("recommender.log"), logging.StreamHandler()],
)
logger = logging.getLogger(__name__)

In [5]:
# data_dir = Path(__file__).parent.parent.parent / "data" / "crawler"
data_dir = Path().resolve().parent.parent / "data" / "crawler"
ratings_files = list(data_dir.glob("boardgame_ratings_*.parquet"))
if not ratings_files:
    raise FileNotFoundError("No ratings files found")

latest_ratings = max(ratings_files, key=lambda x: x.stat().st_mtime)
logger.info(f"Using ratings file: {latest_ratings}")

2025-06-12 11:48:17,441 - __main__ - INFO - Using ratings file: /home/ubuntu/git/pax_tt_recommender/data/crawler/boardgame_ratings_1749439710.parquet


In [7]:
df_ratings = pd.read_parquet(latest_ratings)

In [49]:
df_ratings = pd.read_parquet("../../data/crawler/boardgame_ratings_1749439709.parquet")
df_ratings.shape

(16146, 22)

In [35]:
df_ratings.shape

(1000, 22)

In [78]:
recommender = GameRecommender(min_ratings_per_user=3, exclude_expansions=False)

In [79]:
recommender.fit(df_ratings)

In [82]:
reverse_mappings_path = "../../data/crawler/reverse_mappings_1749439709.json"
with open(reverse_mappings_path, "w") as f:
    json.dump(recommender.reverse_game_mapping, f)

In [83]:
with open(reverse_mappings_path, "r") as f:
    rgm = json.load(f)
gm = {v: k for k, v in rgm.items()}

In [94]:
# game_embeddings = sparse.load_npz("../../data/crawler/game_embeddings_1749439709.npz")
pos_vec = game_embeddings[recommender.game_mapping[224517]].mean(axis=0)

In [96]:
normalize(np.asarray(pos_vec), norm='l2')

array([[0., 0., 0., ..., 0., 0., 0.]], shape=(1, 363464))

In [93]:
from sklearn.preprocessing import normalize

In [85]:
gm

{179350: '0',
 99808: '1',
 286667: '2',
 170041: '3',
 300993: '4',
 139508: '5',
 103651: '6',
 132326: '7',
 171136: '8',
 170202: '9',
 118337: '10',
 11161: '11',
 4282: '12',
 1693: '13',
 193164: '14',
 2387: '15',
 265684: '16',
 174805: '17',
 7204: '18',
 2127: '19',
 1002: '20',
 4047: '21',
 21613: '22',
 28302: '23',
 7308: '24',
 3149: '25',
 255175: '26',
 24082: '27',
 11385: '28',
 27970: '29',
 298619: '30',
 104805: '31',
 409572: '32',
 156457: '33',
 143994: '34',
 3385: '35',
 2173: '36',
 2047: '37',
 147568: '38',
 285554: '39',
 5585: '40',
 366790: '41',
 5651: '42',
 1720: '43',
 164702: '44',
 233956: '45',
 83040: '46',
 209166: '47',
 158564: '48',
 135888: '49',
 220547: '50',
 292907: '51',
 125368: '52',
 144526: '53',
 275996: '54',
 2150: '55',
 2301: '56',
 303676: '57',
 343525: '58',
 216864: '59',
 302477: '60',
 40107: '61',
 229598: '62',
 29351: '63',
 1678: '64',
 187353: '65',
 1713: '66',
 205831: '67',
 89342: '68',
 20436: '69',
 71099: '7

In [68]:
for k,v in enumerate(df_ratings["id"]):
    break

In [70]:
v

179350

In [66]:
rgm

{'0': 0,
 '1': 1,
 '2': 2,
 '3': 3,
 '4': 4,
 '5': 5,
 '6': 6,
 '7': 7,
 '8': 8,
 '9': 9,
 '10': 10,
 '11': 11,
 '12': 12,
 '13': 13,
 '14': 14,
 '15': 15,
 '16': 16,
 '17': 17,
 '18': 18,
 '19': 19,
 '20': 20,
 '21': 21,
 '22': 22,
 '23': 23,
 '24': 24,
 '25': 25,
 '26': 26,
 '27': 27,
 '28': 28,
 '29': 29,
 '30': 30,
 '31': 31,
 '32': 32,
 '33': 33,
 '34': 34,
 '35': 35,
 '36': 36,
 '37': 37,
 '38': 38,
 '39': 39,
 '40': 40,
 '41': 41,
 '42': 42,
 '43': 43,
 '44': 44,
 '45': 45,
 '46': 46,
 '47': 47,
 '48': 48,
 '49': 49,
 '50': 50,
 '51': 51,
 '52': 52,
 '53': 53,
 '54': 54,
 '55': 55,
 '56': 56,
 '57': 57,
 '58': 58,
 '59': 59,
 '60': 60,
 '61': 61,
 '62': 62,
 '63': 63,
 '64': 64,
 '65': 65,
 '66': 66,
 '67': 67,
 '68': 68,
 '69': 69,
 '70': 70,
 '71': 71,
 '72': 72,
 '73': 73,
 '74': 74,
 '75': 75,
 '76': 76,
 '77': 77,
 '78': 78,
 '79': 79,
 '80': 80,
 '81': 81,
 '82': 82,
 '83': 83,
 '84': 84,
 '85': 85,
 '86': 86,
 '87': 87,
 '88': 88,
 '89': 89,
 '90': 90,
 '91': 91,
 '92': 9

In [75]:
df_tmp = df_ratings.set_index("id")
for k,v in df_tmp.iterrows():
    break

In [76]:
k

179350

In [44]:
recommender.rating_matrix

<List of Lists sparse matrix of dtype 'float64'
	with 433290 stored elements and shape (130418, 1000)>

In [77]:
class GameRecommender:
    def __init__(self, min_ratings_per_user: int = 3, exclude_expansions: bool = False):
        """
        Initialize the recommender system.

        Args:
            min_ratings_per_user (int): Minimum number of ratings a user must have to be included
            exclude_expansions (bool): Whether to exclude expansions from recommendations
        """
        self.min_ratings_per_user = min_ratings_per_user
        self.exclude_expansions = exclude_expansions
        self.user_mapping = {}  # Maps user IDs to indices
        self.game_mapping = {}  # Maps game IDs to indices
        self.reverse_game_mapping = {}  # Maps indices back to game IDs
        self.rating_matrix = None
        self.game_embeddings = None
        self.valid_game_ids = (
            None  # Set of valid game IDs (non-expansions if exclude_expansions is True)
        )

    def _create_rating_matrix(self, df: pd.DataFrame) -> sparse.csr_matrix:
        """
        Convert the input dataframe into a sparse rating matrix.

        Args:
            df (pd.DataFrame): Input dataframe with game_id as index and rating columns containing user lists

        Returns:
            sparse.csr_matrix: Sparse matrix of user-game ratings
        """
        # check if the dataframe has a column called "id"
        if "id" in df.columns:
            df = df.set_index("id")

        # Create mappings for users and games
        all_users = set()
        for col in df.columns:
            # Handle both list and non-list values in the ratings
            for users in df[col].dropna():
                if isinstance(users, list):
                    all_users.update(str(user) for user in users)
                else:
                    # If it's a single user, add them directly
                    all_users.add(str(users))

        # Create user mapping
        for i, user in enumerate(sorted(all_users)):
            self.user_mapping[user] = i

        # Create game mapping
        for i, game_id in enumerate(df.index):
            self.game_mapping[game_id] = i
            self.reverse_game_mapping[i] = game_id

#         # Initialize sparse matrix
#         n_users = len(self.user_mapping)
#         n_games = len(self.game_mapping)
#         rating_matrix = sparse.lil_matrix((n_users, n_games))

#         # Fill the rating matrix
#         for game_id, row in df.iterrows():
#             game_idx = self.game_mapping[game_id]
#             for rating, users in row.items():
#                 if rating == "id":  # Skip the id column
#                     continue

#                 # Skip if users is NaN
#                 if users is None:
#                     continue

#                 # Add ratings for each user
#                 for user in users:
#                     if user in self.user_mapping:
#                         user_idx = self.user_mapping[user]
#                         rating_matrix[user_idx, game_idx] = float(rating)

        # Convert to CSR format for efficient operations
        return rating_matrix.tocsr()

    def _filter_users(self, rating_matrix: sparse.csr_matrix) -> sparse.csr_matrix:
        """
        Remove users with fewer than min_ratings_per_user ratings.

        Args:
            rating_matrix (sparse.csr_matrix): Input rating matrix

        Returns:
            sparse.csr_matrix: Filtered rating matrix
        """
        # Count non-zero elements per user (number of ratings)
        user_rating_counts = np.diff(rating_matrix.indptr)

        # Get indices of users with enough ratings
        valid_user_indices = np.where(user_rating_counts >= self.min_ratings_per_user)[
            0
        ]

        # Filter the matrix
        return rating_matrix[valid_user_indices]

    def _load_valid_games(self):
        """Load the set of valid game IDs from the ranks file."""
        if not self.exclude_expansions:
            return None

        # Find the most recent ranks file
        data_dir = Path(__file__).parent.parent.parent / "data" / "crawler"
        ranks_files = list(data_dir.glob("boardgame_ranks_*.csv"))
        if not ranks_files:
            logger.warning("No ranks files found, cannot exclude expansions")
            return None

        latest_ranks = max(ranks_files, key=lambda x: x.stat().st_mtime)
        logger.info(f"Loading valid games from: {latest_ranks}")

        # Read the ranks file and get non-expansion games
        df_ranks = pd.read_csv(latest_ranks, sep="|", escapechar="\\")
        valid_games = df_ranks[df_ranks["is_expansion"] == 0]["id"].tolist()
        return set(str(gid) for gid in valid_games)

    def fit(self, df: pd.DataFrame) -> None:
        """
        Fit the recommender system to the input data.

        Args:
            df (pd.DataFrame): Input dataframe with game_id as index and rating columns containing user lists
        """
        # Create and filter rating matrix
        self.rating_matrix = self._create_rating_matrix(df)
#         self.rating_matrix = self._filter_users(self.rating_matrix)

        # Compute game similarity matrix
#         self.game_embeddings = normalize(self.rating_matrix.T, norm="l2", axis=1)

In [ ]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity
from typing import Dict, List, Tuple
import argparse
import logging
from pathlib import Path
import json

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    handlers=[logging.FileHandler("recommender.log"), logging.StreamHandler()],
)
logger = logging.getLogger(__name__)

class GameRecommender:
    def __init__(self, min_ratings_per_user: int = 3, exclude_expansions: bool = False):
        """
        Initialize the recommender system.
        
        Args:
            min_ratings_per_user (int): Minimum number of ratings a user must have to be included
            exclude_expansions (bool): Whether to exclude expansions from recommendations
        """
        self.min_ratings_per_user = min_ratings_per_user
        self.exclude_expansions = exclude_expansions
        self.user_mapping = {}  # Maps user IDs to indices
        self.game_mapping = {}  # Maps game IDs to indices
        self.reverse_game_mapping = {}  # Maps indices back to game IDs
        self.rating_matrix = None
        self.game_similarity = None
        self.valid_game_ids = None  # Set of valid game IDs (non-expansions if exclude_expansions is True)
        
    def _create_rating_matrix(self, df: pd.DataFrame) -> sparse.csr_matrix:
        """
        Convert the input dataframe into a sparse rating matrix.
        
        Args:
            df (pd.DataFrame): Input dataframe with game_id as index and rating columns containing user lists
            
        Returns:
            sparse.csr_matrix: Sparse matrix of user-game ratings
        """
        # Create mappings for users and games
        all_users = set()
        for col in df.columns:
            if col == 'id':  # Skip the id column
                continue
            # Handle both list and non-list values in the ratings
            for users in df[col].dropna():
                if isinstance(users, list):
                    all_users.update(str(user) for user in users)
                else:
                    # If it's a single user, add them directly
                    all_users.add(str(users))
        
        # Create user mapping
        for i, user in enumerate(sorted(all_users)):
            self.user_mapping[user] = i
            
        # Create game mapping
        for i, game_id in enumerate(df.index):
            self.game_mapping[game_id] = i
            self.reverse_game_mapping[i] = game_id
            
        # Initialize sparse matrix
        n_users = len(self.user_mapping)
        n_games = len(self.game_mapping)
        rating_matrix = sparse.lil_matrix((n_users, n_games))
        
        # Fill the rating matrix
        for game_id, row in df.iterrows():
            game_idx = self.game_mapping[game_id]
            for rating, users in row.items():
                if rating == 'id':  # Skip the id column
                    continue
                    
                # Skip if users is NaN
                if users is None:
                    continue
                    
                # Add ratings for each user
                for user in users:
                    if user in self.user_mapping:
                        user_idx = self.user_mapping[user]
                        rating_matrix[user_idx, game_idx] = float(rating)
        
        # Convert to CSR format for efficient operations
        return rating_matrix.tocsr()
    
    def _filter_users(self, rating_matrix: sparse.csr_matrix) -> sparse.csr_matrix:
        """
        Remove users with fewer than min_ratings_per_user ratings.
        
        Args:
            rating_matrix (sparse.csr_matrix): Input rating matrix
            
        Returns:
            sparse.csr_matrix: Filtered rating matrix
        """
        # Count non-zero elements per user (number of ratings)
        user_rating_counts = np.diff(rating_matrix.indptr)
        
        # Get indices of users with enough ratings
        valid_user_indices = np.where(user_rating_counts >= self.min_ratings_per_user)[0]
        
        # Filter the matrix
        return rating_matrix[valid_user_indices]

    def fit(self, df: pd.DataFrame) -> None:
        """
        Fit the recommender system to the input data.
        
        Args:
            df (pd.DataFrame): Input dataframe with game_id as index and rating columns containing user lists
        """
        # Create and filter rating matrix
        self.rating_matrix = self._create_rating_matrix(df)
        self.rating_matrix = self._filter_users(self.rating_matrix)
        
        # Compute game similarity matrix
        self.game_similarity = cosine_similarity(self.rating_matrix.T)
        
    def recommend_similar_games(self, 
                              game_ids: List[str], 
                              disliked_games: List[str] = None,
                              n_recommendations: int = 5,
                              anti_weight: float = 1.0) -> List[Tuple[str, float]]:
        """
        Generate recommendations based on a list of game IDs, with optional anti-recommendations.
        
        Args:
            game_ids (List[str]): List of game IDs to base recommendations on
            disliked_games (List[str], optional): List of game IDs to use as anti-recommendations
            n_recommendations (int): Number of recommendations to generate
            anti_weight (float): Weight to apply to anti-recommendations
            
        Returns:
            List[Tuple[str, float]]: List of (game_id, similarity_score) tuples
        """
        # Convert liked game IDs to indices
        game_indices = []
        for game_id in game_ids:
            if game_id in self.game_mapping:
                game_indices.append(self.game_mapping[game_id])
        
        if not game_indices:
            return []
            
        # Calculate average similarity scores for all games
        avg_similarities = np.zeros(len(self.game_mapping))
        for game_idx in game_indices:
            avg_similarities += self.game_similarity[game_idx]
        avg_similarities /= len(game_indices)
        
        # Apply anti-recommendations if provided
        if disliked_games:
            disliked_indices = []
            for game_id in disliked_games:
                if game_id in self.game_mapping:
                    disliked_indices.append(self.game_mapping[game_id])
            
            if disliked_indices:
                # Calculate average similarity to disliked games
                anti_similarities = np.zeros(len(self.game_mapping))
                for game_idx in disliked_indices:
                    anti_similarities += self.game_similarity[game_idx]
                anti_similarities /= len(disliked_indices)
                
                # Subtract anti-similarities from the main similarities
                avg_similarities -= anti_similarities * anti_weight
        
        # Remove the input games from recommendations
        for game_idx in game_indices:
            avg_similarities[game_idx] = -1
        if disliked_games:
            for game_idx in disliked_indices:
                avg_similarities[game_idx] = -1

        # Exclude expansions if configured
        if self.exclude_expansions and self.valid_game_ids:
            for idx, game_id in self.reverse_game_mapping.items():
                if game_id not in self.valid_game_ids:
                    avg_similarities[idx] = -1
            
        # Get top N similar games
        top_indices = np.argsort(avg_similarities)[-n_recommendations:][::-1]
        
        return [(self.reverse_game_mapping[idx], avg_similarities[idx]) 
                for idx in top_indices if avg_similarities[idx] > 0]





In [46]:
game_embeddings = sparse.load_npz("../../data/crawler/game_embeddings_1749439709.npz")

# String matching

In [41]:
from rapidfuzz import process, fuzz
import unicodedata
import re

In [45]:
df_games.columns

Index(['id', 'name', 'thumbnail', 'image', 'description', 'yearpublished',
       'rank', 'usersrated', 'is_expansion', 'abstracts_rank', 'cgs_rank',
       'childrensgames_rank', 'familygames_rank', 'partygames_rank',
       'strategygames_rank', 'thematic_rank', 'wargames_rank', 'minplayers',
       'maxplayers', 'playingtime', 'minplaytime', 'maxplaytime', 'minage',
       'owned', 'trading', 'wanting', 'wishing', 'numcomments', 'numweights',
       'numratings', 'bayesaverage', 'average', 'stddev', 'median',
       'averageweight'],
      dtype='object')

In [ ]:
processed_games_data_1749253652

In [81]:
df_games_raw = pd.read_csv(
    "../../data/processed/processed_games_data_1750514842.csv",
    sep="|",
    escapechar="\\",
    quoting=csv.QUOTE_NONE,
    usecols=["id", "name", "rank", "usersrated", "yearpublished"],
)
df_games = df_games_raw.copy()
df_games["name"] = df_games["name"].str.lower()
for col in ["id", "rank", "usersrated", "yearpublished"]:
    df_games[col] = df_games[col].astype(pd.Int64Dtype())
df_games.head()

,id,name,yearpublished,rank,usersrated
0,224517,brass: birmingham,2018,1,52086
1,161936,pandemic legacy: season 1,2015,2,55588
2,342942,ark nova,2021,3,52459
3,174430,gloomhaven,2017,4,64874
4,233078,twilight imperium: fourth edition,2017,5,26087


In [218]:
df_pax = pd.read_csv("../../data/pax/pax_tt_games_unplugged_2024.csv", sep="|")
df_pax["bgg_id"] = df_pax["bgg_id"].astype(pd.Int64Dtype())
# games which do not exist in bgg
drop_games = [3708, 5145]
df_pax = df_pax.loc[~(df_pax["min_titles_id"].isin(drop_games))].reset_index(drop=True)
print(df_pax.shape)
display(df_pax.isna().sum())
df_pax.head()

(1831, 9)


name                 0
name_raw          1805
bgg_id            1240
publisher            1
min_titles_id        0
titles_id_list       0
convention           0
year                 0
year_added           0
dtype: int64

,name,name_raw,bgg_id,publisher,min_titles_id,titles_id_list,convention,year,year_added
0,#artbox,NaN,<NA>,lavka games,5025,5025,unplugged,2024,2024
1,1001 islands,NaN,<NA>,ludonaute,4036,4036,unplugged,2024,2022
2,13 beavers,NaN,<NA>,format games,4996,4996,unplugged,2024,2024
3,18lilliput,NaN,<NA>,fox in the box,2808,2808,unplugged,2024,2019
4,1902 melies,NaN,<NA>,looping games,4539,4539,unplugged,2024,2023


In [219]:
dupes = {
    0: ["ahoy", 359402, 4146],
    1: ["akropolis", 357563, 4311],
    2: ["arctic", 420214, 5019],
    3: ["autobahn", 348554, 4649],
    4: ["babylon", 418354, 5038],
    5: ["blitzkrieg!: world war two in 20 minutes", 258210, 3848],
    6: ["bohnanza", 11, 3724],
    7: ["bonsai", 366994, 4506],
    8: ["can of worms", 417472, 4964],
    9: ["canvas", 290236, 3879],
    10: ["cheese factory", 365551, 4599],
    11: ["citadels", 478, 1857],
    12: ["cities", 402676, 4862],
    13: ["combo", 394887, 5135],
    14: ["come together", 361025, 4274],
    15: ["cosmic encounter", 39463, 1841],
    16: ["decrypto", 225694, 2148],
    17: ["deep dive", 376747, 4457],
    18: ["deities", 366575, 4326],
    19: ["dimension", 153318, 1411],
    20: ["dinosaur world", 317457, 3841],
    21: ["dodo", 295490, 5070],
    22: ["equinox", 329450, 3588],
    23: ["escape from new york", 362723, 4893],
    24: ["evergreen", 363307, 4239],
    25: ["explorers", 330174, 4750],
    26: ["express", 379, 5010],
    27: ["faraway", 385761, 4687],
    28: ["festival", 413948, 5045],
    29: ["finca", 40628, 4988],
    30: ["floristry", 376218, 4329],
    31: ["footprints", 386271, 4535],
    32: ["glasgow", 301716, 3738],
    33: ["golem", 298383, 3891],
    34: ["hack & slash", 322250, 3670],
    35: ["hamlet", 276086, 4295],
    36: ["high society", 220, 5064],
    37: ["hit", 406706, 4898],
    38: ["illiterati", 348602, 4709],
    39: ["illuminati", 316093, 3673],
    40: ["intarsia", 422126, 4833],
    41: ["islet", 392473, 4511],
    42: ["jamaica", 28023, 3864],
    43: ["journey to the center of the earth", 324246, 3830],
    44: ["kapow!", 380845, 4264],
    45: ["l.l.a.m.a.", 266083, 3589],
    46: ["landmarks", 421374, 4801],
    47: ["last will", 97842, 4941],
    48: ["linkee!", 147194, 3680],
    49: ["lynx", 415376, 5092],
    50: ["maharaja", 298378, 5027],
    51: ["metro", 559, 1907],
    52: ["mooncake master", 400265, 5122],
    53: ["mycelia", 392023, 4636],
    54: ["niagara", 13308, 2741],
    55: ["nice buns", 337391, 3674],
    56: ["obsession", 231733, 4655],
    57: ["onslaught", 4287, 4228],
    58: ["outlines", 366981, 4072],
    59: ["panorama", 352293, 4257],
    60: ["papillon", 241066, 3336],
    61: ["pass", 402109, 4490],
    62: ["picture perfect", 299963, 4148],
    63: ["planetoid", 232201, 3001],
    64: ["plutocracy", 363531, 4335],
    65: ["portals", 386168, 4656],
    66: ["push", 265256, 4025],
    67: ["quicksand", 396655, 4496],
    68: ["rapa nui", 314401, 3773],
    69: ["run", 368028, 4708],
    70: ["san francisco", 357271, 4123],
    71: ["scout", 291453, 3793],
    72: ["skull", 92415, 3642],
    73: ["slanguage", 419668, 4971],
    74: ["sol", 192286, 2872],
    75: ["sorry!", 2407, 2721],
    76: ["space base", 242302, 2606],
    77: ["spotlight", 420876, 4921],
    78: ["spots", 372559, 4115],
    79: ["star realms", 147020, 4422],
    80: ["stella", 263214, 3631],
    81: ["strike", 123570, 4397],
    82: ["tales of the arabian nights", 34119, 1798],
    83: ["tatsu", 369084, 4781],
    84: ["terminus", 350198, 4837],
    85: ["tesseract", 367385, 4613],
    86: ["tetris", 342016, 4752],
    87: ["the game", 173090, 3659],
    88: ["the gang", 411567, 5051],
    89: ["the hunt", 376223, 4692],
    90: ["the lord of the rings", 823, 3261],
    91: ["trash talk", 387219, 4658],
    92: ["tribes", 258104, 3591],
    93: ["trio", 352515, 4732],
    94: ["unearth", 217085, 1632],
    95: ["unfair", 179172, 1686],
    96: ["vendetta", 336791, 5029],
    97: ["vintage", 296667, 3703],
    98: ["vivarium", 361240, 4338],
    99: ["waggle dance", 338351, 3983],
    100: ["watergate", 274364, 2980],
    101: ["whitewater", 93563, 3165],
    102: ["word slam", 203411, 1648],
    103: ["bar fight", 323508, 4164],
    104: ["mycelium", 371091, 4618],
    105: ["wing it", 234274, 5142],
    106: ["compile", 406652, 4909],
    107: ["yokai", 269146, 3965],
    108: ["incognito", 406863, 5030],
    109: ["rapture", 351332, 5033],
    110: ["bounty hunters", 410655, 5006],
    111: ["seti", 418059, 5104],
}
df_dupes = pd.DataFrame().from_dict(dupes, orient="index")
df_dupes.columns = ["name", "bgg_id_fill", "min_titles_id"]
df_dupes["bgg_id_fill"] = df_dupes["bgg_id_fill"].astype(pd.Int64Dtype())

In [221]:
df_pax_fill1 = df_pax.merge(df_dupes[["min_titles_id", "bgg_id_fill"]], on=["min_titles_id"], how="left")
df_pax_fill1.loc[df_pax_fill1["bgg_id"].isna(), "bgg_id"] = df_pax_fill1["bgg_id_fill"]
df_pax_fill1 = df_pax_fill1.drop(columns=["bgg_id_fill"])
print(df_pax_fill1.shape)
display(df_pax_fill1.isna().sum())
df_pax_fill1.head()

(1831, 9)


name                 0
name_raw          1805
bgg_id            1131
publisher            1
min_titles_id        0
titles_id_list       0
convention           0
year                 0
year_added           0
dtype: int64

,name,name_raw,bgg_id,publisher,min_titles_id,titles_id_list,convention,year,year_added
0,#artbox,NaN,<NA>,lavka games,5025,5025,unplugged,2024,2024
1,1001 islands,NaN,<NA>,ludonaute,4036,4036,unplugged,2024,2022
2,13 beavers,NaN,<NA>,format games,4996,4996,unplugged,2024,2024
3,18lilliput,NaN,<NA>,fox in the box,2808,2808,unplugged,2024,2019
4,1902 melies,NaN,<NA>,looping games,4539,4539,unplugged,2024,2023


In [222]:
query_time = int(time())
df_pax_fill1.to_csv(
    f"../../data/pax/pax_tt_games_{query_time}.csv",
    index=False,
    sep="|",
    escapechar="\\",
    quoting=csv.QUOTE_NONE,
)

In [223]:
df_merge_games = df_pax_fill1.loc[df_pax_fill1["bgg_id"].isna()].merge(df_games, on="name", how="left")
print(df_merge_games.isna().sum())
df_merge_games.head()

name                 0
name_raw          1131
bgg_id            1131
publisher            0
min_titles_id        0
titles_id_list       0
convention           0
year                 0
year_added           0
id                 244
yearpublished      244
rank               315
usersrated         244
decode             244
dtype: int64


,name,name_raw,bgg_id,publisher,min_titles_id,titles_id_list,convention,year,year_added,id,yearpublished,rank,usersrated,decode
0,#artbox,NaN,<NA>,lavka games,5025,5025,unplugged,2024,2024,<NA>,<NA>,<NA>,<NA>,NaN
1,1001 islands,NaN,<NA>,ludonaute,4036,4036,unplugged,2024,2022,351972,2022,9549,175,1001 islands
2,13 beavers,NaN,<NA>,format games,4996,4996,unplugged,2024,2024,423850,2024,12021,153,13 beavers
3,18lilliput,NaN,<NA>,fox in the box,2808,2808,unplugged,2024,2019,250621,2018,3237,956,18lilliput
4,1902 melies,NaN,<NA>,looping games,4539,4539,unplugged,2024,2023,<NA>,<NA>,<NA>,<NA>,NaN


In [224]:
# check for dupes
df_merge_games.loc[
    df_merge_games.duplicated(subset=["name"], keep=False),
    [
        "min_titles_id",
        "titles_id_list",
        "name",
        "id",
        "name_raw",
        "publisher",
        "year_added",
        "yearpublished",
        "rank",
        "usersrated",
    ],
].sort_values(["name", "usersrated", "rank"], ascending=[True, False, True])

,min_titles_id,titles_id_list,name,id,name_raw,publisher,year_added,yearpublished,rank,usersrated


In [164]:
df_merge_games.loc[
    df_merge_games["id"].notna(),
    [
        "name",
        "id",
        "min_titles_id",
        "name_raw",
        "bgg_id",
        "publisher",
        "titles_id_list",
        "convention",
        "year",
        "year_added",
        "yearpublished",
        "rank",
        "usersrated",
    ],
].sort_values("usersrated").iloc[300:400]

,name,id,min_titles_id,name_raw,bgg_id,publisher,titles_id_list,convention,year,year_added,yearpublished,rank,usersrated
941,the coldest night,300727,4751,NaN,<NA>,indie boards & cards,4751,unplugged,2024,2024,2020,10207,291
762,rivages,409581,4983,NaN,<NA>,pandasaurus,4983,unplugged,2024,2024,2024,6340,294
854,stalk exchange,416552,4820,NaN,<NA>,the op games,4820,unplugged,2024,2024,2024,9226,297
777,sabobatage,342636,4673,NaN,<NA>,boba biz,4673,unplugged,2024,2023,2021,11453,297
1081,war story: occupied france,408381,5112,NaN,<NA>,osprey games,5112,unplugged,2024,2024,2024,4647,300
602,mouse guard: swords & strongholds,162315,1371,NaN,<NA>,burning wheel,1371,unplugged,2024,2016,2015,6920,300
911,sync or swim,353196,4113,NaN,<NA>,bezier games,4113,unplugged,2024,2022,2022,6410,301
958,the impossible machine,95893,402,NaN,<NA>,closet nerd games,402,unplugged,2024,2014,2011,26670,301
464,jokkmokk: the winter market,388570,4633,NaN,<NA>,wizk!ds,4633,unplugged,2024,2023,2023,6066,301
1045,umbra via,326400,3634,NaN,<NA>,pandasaurus,3634,unplugged,2024,2021,2021,8566,309


bar fight	323508	4164	
mycelium	371091	4618	
wing it	234274	5142
compile	406652	4909	
yokai	269146	3965
incognito	406863	5030
rapture	351332	5033
bounty hunters	410655	5006	
seti	418059	5104	

In [145]:
df_merge.loc[(df_merge["bgg_id"].notna()) & (df_merge["id"].notna()) & (df_merge["id"]!= df_merge["bgg_id"])]

,name,name_raw,bgg_id,publisher,min_titles_id,titles_id_list,convention,year,year_added,id,yearpublished,rank,usersrated


In [225]:
df_merge_games_filled = df_merge_games.loc[df_merge_games["id"].notna(), ["min_titles_id", "id"]]
df_pax_fill2 = df_pax_fill1.merge(df_merge_games_filled, on=["min_titles_id"], how="left")
df_pax_fill2.loc[df_pax_fill2["bgg_id"].isna(), "bgg_id"] = df_pax_fill2["id"]
df_pax_fill2 = df_pax_fill2.drop(columns=["id"])
print(df_pax_fill2.shape)
display(df_pax_fill2.isna().sum())

(1831, 9)


name                 0
name_raw          1805
bgg_id             244
publisher            1
min_titles_id        0
titles_id_list       0
convention           0
year                 0
year_added           0
dtype: int64

In [226]:
query_time = int(time())
df_pax_fill2.to_csv(f"../../data/pax/pax_tt_games_{query_time}.csv", index=False,sep="|", escapechar="\\", quoting=csv.QUOTE_NONE)

In [173]:
df_games["name"].apply(lambda x: unicodedata.normalize('NFKD', x))

0                         brass: birmingham
1                 pandemic legacy: season 1
2                                  ark nova
3                                gloomhaven
4         twilight imperium: fourth edition
                        ...                
129542                            word bank
129543                             floriado
129544                         claw machine
129545    sonic the hedgehog: battle racers
129546                                jaliz
Name: name, Length: 129547, dtype: object

In [227]:
df_pax_null = df_pax_fill2.loc[df_pax_fill2["bgg_id"].isna()].copy()
df_pax_null

,name,name_raw,bgg_id,publisher,min_titles_id,titles_id_list,convention,year,year_added
0,#artbox,NaN,<NA>,lavka games,5025,5025,unplugged,2024,2024
4,1902 melies,NaN,<NA>,looping games,4539,4539,unplugged,2024,2023
19,9 peak,NaN,<NA>,manifest destiny,4321,4321,unplugged,2024,2022
21,a game of folklore,NaN,<NA>,kenechukwu,3762,3762,unplugged,2024,2021
30,adventure party,NaN,<NA>,smirk & dagger,5080,5080,unplugged,2024,2024
34,aggretsuko,NaN,<NA>,renegade game studios,3940,3940,unplugged,2024,2022
36,agueda city of umbrellas,NaN,<NA>,25th century games,4847,4847,unplugged,2024,2024
49,alpha beasts attack!,NaN,<NA>,origames,5124,5124,unplugged,2024,2024
83,ascension 10 year anniversary edition,NaN,<NA>,stoneblade entertainment,3623,3623,unplugged,2024,2021
86,ascension: deckbuilding game year six collecto...,NaN,<NA>,stoneblade entertainment,3626,3626,unplugged,2024,2021


In [188]:
# Assume these columns:
# df_pax["name"]        → names to match (subset)
# df_games["name"]      → reference names
# df_games["id"]        → what we want to retrieve

# Step 1: Create a function to match each pax name to the best game name
def get_best_match(pax_name, game_names, scorer=fuzz.token_sort_ratio, threshold=80):
    match = process.extractOne(pax_name, game_names, scorer=scorer)
    if match and match[1] >= threshold:
        return match[0]  # matched name
    return None  # no good match


# Step 2: Apply matching logic
df_pax_null["matched_name"] = df_pax_null["name"].apply(lambda x: get_best_match(x, df_games["name"]))

# # Step 3: Merge to get IDs from df_games
df_pax_null_merged = df_pax_null.merge(df_games[["id", "name", "usersrated"]], left_on="matched_name", right_on="name", how="left", suffixes=('', '_matched'))

# # Optional: Keep only relevant columns
# df_result = df_merged[["name", "matched_name", "id"]]

In [198]:
df_pax_null_merged[
    [
        "name",
        "id",
        "min_titles_id",
        "matched_name",
        "name_raw",
        "usersrated",
        "bgg_id",
        "publisher",
        "titles_id_list",
        "convention",
        "year",
        "year_added",
        "name_matched",
    ]
]

,name,id,min_titles_id,matched_name,name_raw,usersrated,bgg_id,publisher,titles_id_list,convention,year,year_added,name_matched
0,#artbox,280304,5025,artbox,NaN,392,<NA>,lavka games,5025,unplugged,2024,2024,artbox
1,1902 melies,393456,4539,1902 méliès,NaN,281,<NA>,looping games,4539,unplugged,2024,2023,1902 méliès
2,9 peak,375388,4321,9peak,NaN,1,<NA>,manifest destiny,4321,unplugged,2024,2022,9peak
3,a game of folklore,<NA>,3762,None,NaN,<NA>,<NA>,kenechukwu,3762,unplugged,2024,2021,NaN
4,adventure party,15240,5080,adventure park,NaN,0,<NA>,smirk & dagger,5080,unplugged,2024,2024,adventure park
5,aggretsuko,<NA>,3940,None,NaN,<NA>,<NA>,renegade game studios,3940,unplugged,2024,2022,NaN
6,agueda city of umbrellas,<NA>,4847,None,NaN,<NA>,<NA>,25th century games,4847,unplugged,2024,2024,NaN
7,alpha beasts attack!,<NA>,5124,None,NaN,<NA>,<NA>,origames,5124,unplugged,2024,2024,NaN
8,ascension 10 year anniversary edition,325810,3623,ascension: 10 year anniversary edition,NaN,951,<NA>,stoneblade entertainment,3623,unplugged,2024,2021,ascension: 10 year anniversary edition
9,ascension: deckbuilding game year six collecto...,341554,3626,ascension year six collector's edition,NaN,52,<NA>,stoneblade entertainment,3626,unplugged,2024,2021,ascension year six collector's edition


In [228]:
str_match_1 = {
    0: ["#artbox", 280304, 5025],
    1: ["1902 melies", 393456, 4539],
    2: ["9 peak", 375388, 4321],
    3: ["ascension 10 year anniversary edition", 325810, 3623],
    4: ["astro knights eternity", 386538, 4743],
    5: ["bestiary of sigillium collectors edition", 366577, 4677],
    6: ["blast track!", 417492, 4974],
    7: ["boop the halls", 417696, 5078],
    8: ["boxtop pinball haunted house", 417629, 4989],
    9: ["burn the port", 393346, 4860],
    10: ["cafe del gatto", 377738, 4530],
    11: ["cafe international", 214, 3319],
    12: ["cake auf", 343848, 3992],
    13: ["canopy evergreen", 391886, 5085],
    14: ["cards against humanity mini", 50381, 4955],
    15: ["cascadia: rolling hils", 407317, 5137],
    16: ["catan + 5-6 player + seafarers + seafarers 5-6...", 13, 4430],
    17: ["catan + 5-6 players + cities & knights + 5-6 c...", 13, 4431],
    18: ["catan + cities & knights + legend of the conqu...", 13, 4426],
    19: ["catan + cities & knights + seafarers + treasur...", 13, 4429],
    20: ["catan + explorers & pirates expansion", 13, 4427],
    21: ["catan + traders and barbarians", 13, 4428],
    22: ["catan new energies", 408727, 5056],
    23: ["catan starfarers duel", 383481, 5074],
    24: ["chronicles of light: darkness falls", 411369, 4884],
    25: ["clank! catacombs", 365717, 4370],
    26: ["cloud nine", 381166, 4828],
    27: ["codenames duet", 224037, 4938],
    28: ["codenames duet xxl", 224037, 4943],
    29: ["codenames pictures xxl", 198773, 2943],
    30: ["codenames xxl", 178900, 2942],
    31: ["codenames: disney family edition", 220775, 1952],
    32: ["corrupt parliament", 374147, 4345],
    33: ["cthullhu - death may die", 253344, 3093],
    34: ["cultists & cthulhu 2nd edition", 410254, 4952],
    35: ["cyberpunk 2077", 352138, 5005],
    36: ["dance of the ibexes", 13005, 4896],
    37: ["dango bingo", 374953, 4327],
    38: ["decorum", 344554, 3979],
    39: ["disney jungle cruise adventure games", 303650, 4016],
    40: ["dro polter", 405537, 4712],
    41: ["dune imperium", 316554, 4018],
    42: ["dune imperium uprising", 397598, 4606],
    43: ["ebb and flow", 381524, 4617],
    44: ["everyone thinks this game is awesome", 344123, 4660],
    45: ["exit the game kids: riddles in monsterville", 376479, 5054],
    46: ["fabeland", 361909, 4076],
    47: ["farcry: escape from rook islands", 347935, 4894],
    48: ["fire & stone: siege of vienna 1863", 362868, 4054],
    49: ["five tribes", 157354, 1204],
    50: ["flugelrad", 400097, 4544],
    51: ["future inc", 387305, 4515],
    52: ["gabanga!", 394889, 5133],
    53: ["general orders world war ii", 381435, 5032],
    54: ["groo the game", 194, 5011],
    55: ["her story", 369436, 4421],
    56: ["honshu", 207336, 3099],
    57: ["horrified greek monsters", 397897, 4662],
    58: ["ilimat", 210040, 5148],
    59: ["imperium horizons", 367518, 5040],
    60: ["infiltrators", 353945, 4493],
    61: ["islebound emerald edition", 401175, 5055],
    62: ["it's kind of a fun story family edition", 413555, 5126],
    63: ["khora: rise of an empire", 299684, 3818],
    64: ["king of tokyo duel", 422484, 4932],
    65: ["king of tokyo origins", 403240, 4933],
    66: ["kombo klash", 317312, 4216],
    67: ["kronologic", 402111, 4806],
    68: ["ku-ka-king", 402208, 4525],
    69: ["kutna hora: the city of silver", 385610, 4500],
    70: ["legends of andor eternal frost", 363086, 4438],
    71: ["long shot the dice game", 295374, 5089],
    72: ["lord of the rings adventure to mount doom", 366278, 4437],
    73: ["magic maze: kids", 242740, 4770],
    74: ["magic: the gathering - explorers of ixalan", 231309, 4065],
    75: ["marvel dice throne: captain marvel vs black panther", 360061, 4367],
    76: ["merchants and marauders", 25292, 3145],
    77: ["micromacro crime city - all in", 364766, 4034],
    78: ["middara:unintentional malum act 1", 169427, 3867],
    79: ["mind mgmt: the psychic espionage game", 284653, 3823],
    80: ["mind up", 377515, 4986],
    81: ["minecraft builders & biomes", 269603, 4027],
    82: ["mistborn: the deck building game", 422780, 5153],
    83: ["mlem space agency", 387378, 4507],
    84: ["moon shine empire", 264858, 4592],
    85: ["munchkin the nightmare before christmas", 180816, 1567],
    86: ["murray: the a-hole frog", 416494, 4859],
    87: ["my city roll and build", 351476, 4616],
    88: ["my hero academia plus ultra! board game", 361208, 3905],
    89: ["naruto ninja arena", 320136, 4745],
    90: ["neuro riders", 350992, 4863],
    91: ["nights of fire battle for budapest", 236125, 4350],
    92: ["nunatak temple of ice", 383158, 5057],
    93: ["oh my orchids!", 378556, 5127],
    94: ["ole! guacamole", 341315, 4068],
    95: ["paleo vet", 328643, 4590],
    96: ["pandemic: hot zone europe", 329670, 3808],
    97: ["pandemic: hot zone north america", 301919, 3618],
    98: ["penny arcade: paint the line ebc - red tide", 127048, 4170],
    99: ["pest: light in the dark", 395581, 4888],
    100: ["phantom ink arcana", 421873, 5141],
    101: ["potion slingers", 349488, 4226],
    102: ["rallyman dirt", 312959, 4924],
    103: ["re;act", 324403, 5099],
    104: ["rhino hero super battle", 218333, 3624],
    105: ["sankore: the pride of mansa musa", 382518, 5077],
    106: ["shop until you drop lah!", 410275, 5125],
    107: ["sleeping gods primeval peril", 362205, 5059],
    108: ["sleeping queens 2: the rescue", 358050, 4393],
    109: ["so, you've been eaten", 284842, 4331],
    110: ["sour patch kids", 433347, 5007],
    111: ["star wars imperial assault", 164153, 4158],
    112: ["star wars: jabba's palace - a love letter game", 353470, 3899],
    113: ["stich fur stich", 398772, 4529],
    114: ["string railway collection", 391720, 5017],
    115: ["strinking flint", 407548, 4876],
    116: ["super skill pinball: ramp it up!", 328826, 3734],
    117: ["sushi go! spin some for dim sum", 392611, 4765],
    118: ["sweet and spicy", 355737, 4902],
    119: ["talisman: rrevised 4th edition", 27627, 3122],
    120: ["the art project", 378356, 4552],
    121: ["the lord of the rings duel for middle-earth", 421006, 5000],
    122: [
        "the lord of the rings the fellowship of the ring trick-taking game",
        429293,
        5014,
    ],
    123: ["the wandering towers", 355483, 4270],
    124: ["timezoo", 404842, 4918],
    125: ["tir na nog", 343433, 4850],
    126: ["unadunted normandy", 268864, 5060],
    127: ["undaunted battle of britain", 366495, 5050],
    128: ["undaunted north africa", 290359, 5116],
    129: ["vampire the masquerade - rivals expandable card game", 315221, 3935],
    130: ["varmints", 368852, 5147],
    131: ["vs system: the marvel battles", 178892, 4062],
    132: ["where's my ride", 374789, 4630],
    133: ["word traveller", 400508, 5016],
    134: ["xia legends of a drift system", 82222, 3206],
}
df_str_1 = pd.DataFrame().from_dict(str_match_1, orient="index")
df_str_1.columns = ["name", "bgg_id_fill", "min_titles_id"]
df_str_1["bgg_id_fill"] = df_str_1["bgg_id_fill"].astype(pd.Int64Dtype())

In [229]:
df_pax_fill3 = df_pax_fill2.merge(df_str_1[["min_titles_id", "bgg_id_fill"]], on=["min_titles_id"], how="left")
df_pax_fill3.loc[df_pax_fill3["bgg_id"].isna(), "bgg_id"] = df_pax_fill3["bgg_id_fill"]
df_pax_fill3 = df_pax_fill3.drop(columns=["bgg_id_fill"])
print(df_pax_fill3.shape)
display(df_pax_fill3.isna().sum())
df_pax_fill3.head()

(1831, 9)


name                 0
name_raw          1805
bgg_id             109
publisher            1
min_titles_id        0
titles_id_list       0
convention           0
year                 0
year_added           0
dtype: int64

,name,name_raw,bgg_id,publisher,min_titles_id,titles_id_list,convention,year,year_added
0,#artbox,NaN,280304,lavka games,5025,5025,unplugged,2024,2024
1,1001 islands,NaN,351972,ludonaute,4036,4036,unplugged,2024,2022
2,13 beavers,NaN,423850,format games,4996,4996,unplugged,2024,2024
3,18lilliput,NaN,250621,fox in the box,2808,2808,unplugged,2024,2019
4,1902 melies,NaN,393456,looping games,4539,4539,unplugged,2024,2023


In [231]:
query_time = int(time())
df_pax_fill3.to_csv(f"../../data/pax/pax_tt_games_{query_time}.csv", index=False,sep="|", escapechar="\\", quoting=csv.QUOTE_NONE)

In [234]:
df_pax_fill3.loc[
    df_pax_fill3["bgg_id"].isna(),
    [
        "name",
        "bgg_id",
        "min_titles_id",
        "name_raw",
        "publisher",
        "titles_id_list",
        "convention",
        "year",
        "year_added",
    ],
]

,name,bgg_id,min_titles_id,name_raw,publisher,titles_id_list,convention,year,year_added
21,a game of folklore,<NA>,3762,NaN,kenechukwu,3762,unplugged,2024,2021
30,adventure party,<NA>,5080,NaN,smirk & dagger,5080,unplugged,2024,2024
34,aggretsuko,<NA>,3940,NaN,renegade game studios,3940,unplugged,2024,2022
36,agueda city of umbrellas,<NA>,4847,NaN,25th century games,4847,unplugged,2024,2024
49,alpha beasts attack!,<NA>,5124,NaN,origames,5124,unplugged,2024,2024
86,ascension: deckbuilding game year six collecto...,<NA>,3626,NaN,stoneblade entertainment,3626,unplugged,2024,2021
98,avatar: aang's destiny,<NA>,4818,NaN,the op games,4818,unplugged,2024,2024
111,bananya,<NA>,4730,NaN,japanime games,4730,unplugged,2024,2024
116,battalion,<NA>,5098,NaN,osprey games,5098,unplugged,2024,2024
128,beppo,<NA>,4126,NaN,huch!,"4126,4154",unplugged,2024,2022


In [235]:
final_fill = {
    0: ["adventure party", 380106, 5080],
    1: ["aggretsuko", 336206, 3940],
    2: ["agueda city of umbrellas", 317515, 4847],
    3: ["ascension: deckbuilding game year six collecto...", 341554, 3626],
    4: ["avatar: aang's destiny", 416284, 4818],
    5: ["bananya", 344177, 4730],
    6: ["battalion", 415838, 5098],
    7: ["beppo", 27948, 4126],
    8: ["binho", 301480, 4838],
    9: ["bloc by bloc", 356768, 4186],
    10: ["blood and white rose", 400566, 4479],
    11: ["broken and beautiful", 359974, 4297],
    12: ["bumuntu", 201006, 2582],
    13: ["business walrus", 344454, 4734],
    14: ["cars wars sixth edition", 291828, 5003],
    15: ["cascadia + landmarks", 295947, 5021],
    16: ["creepshow", 422535, 4857],
    17: ["critters at war", 247367, 4392],
    18: ["cthulhu!", 149322, 2575],
    19: ["deep rock galactic", 348220, 5103],
    20: ["disney villainous: star wars", 358800, 4020],
    21: ["dorf romantik: the duel", 395364, 4509],
    22: ["dreamworld", 375215, 5001],
    23: ["epic seven arise", 320829, 4773],
    24: ["eris abacus", 437732, 4683],
    25: ["etariya", 369984, 4339],
    26: ["evolution: new world with butterfly effect exp...", 344992, 4407],
    27: ["flatter me - a compliment battle card game", 308696, 4827],
    28: ["fluxx dice (expansion)", 177946, 4716],
    29: ["freelancers", 383206, 4570],
    30: ["frozen shinies", 410662, 4950],
    31: ["game of ham", 270638, 4873],
    32: ["genshin tarot", 346643, 4596],
    33: ["gloom: unfortunate expeditions", 12692, 4625],
    34: ["grandblue fantasy", 421215, 5119],
    35: ["greed dice game", 4282, 4842],
    36: ["hapsburg eclipse", 149361, 4566],
    37: ["harrow county", 360899, 4961],
    38: ["heroes of fairytail", 346958, 4740],
    39: ["heroes welcome: merchants of dragon reach", 212839, 4191],
    40: ["high treason", 197072, 4568],
    41: ["hinata origuchi's seven prophecies", 260568, 4812],
    42: ["holotype", 358974, 4553],
    43: ["hurt party", 413314, 4956],
    44: ["imhotep: a new dynasty", 229839, 2394],
    45: ["ivion", 222291, 4621],
    46: ["joyride duel", 399318, 4895],
    47: ["katmai", 419908, 5100],
    48: ["kcaloria 01 (east texas piney woods)", 375108, 4803],
    49: ["kcaloria 02 (mexican cloud forest)", 375108, 4804],
    50: ["keystone north america 2nd edition - with coas...", 420539, 5018],
    51: ["ko-op", 349091, 4163],
    52: ["labyrinth: disney", 364787, 4001],
    53: ["legend of the five rings: the battle of beiden...", 105099, 4710],
    54: ["lost for words", 357030, 4831],
    55: ["lost kingdom", 288251, 3757],
    56: ["lum lum party", 335733, 3790],
    57: ["marvel villainous", 302336, 4008],
    58: ["merchants of magick", 321641, 3958],
    59: ["merv", 306040, 5047],
    60: ["micromacro full house", 338834, 3820],
    61: ["mieze katze", 424442, 4993],
    62: ["my lil' everdell", 375852, 4165],
    63: ["my little pony - adventures in equestria", 358558, 4355],
    64: ["nasi lemak", 425636, 5061],
    65: ["nasi lemak with the rendang expansion", 425636, 5069],
    66: ["nunatak", 383158, 4502],
    67: ["oath: chronicles of empire & exile", 291572, 3761],
    68: ["oltree", 293835, 4081],
    69: ["on the underground: london/berlin", 281152, 4612],
    70: ["out of line", 426385, 4963],
    71: ["ozark", 367569, 4096],
    72: ["panda trouble", 398996, 5072],
    73: ["planted", 365104, 4394],
    74: ["posthuman saga + the journey home expansion", 244820, 4368],
    75: ["rival restaurants with back for seconds expansion", 262114, 4705],
    76: ["rock hard", 402679, 4865],
    77: ["sansa teutonica", 286749, 4153],
    78: ["scoundrels of skullport", 134342, 2956],
    79: ["shadows", 371539, 5031],
    80: ["shamanz", 313262, 4091],
    81: ["skyrockets", 398684, 4609],
    82: ["slay the spire", 338960, 4822],
    83: ["spectral: a deduction game", 388476, 4858],
    84: ["squid game", 367512, 4129],
    85: ["super skill pinball 4-cade", 302524, 4108],
    86: ["the plot thickens", 210816, 4945],
    87: ["the quacks of quedlinburg", 244521, 4152],
    88: ["the quest for el dorado with heroes & hexes", 217372, 2920],
    89: ["undaunted 2200", 401978, 5068],
    90: ["undaunted reinforcements", 313441, 5117],
    91: ["vast: mysterious manor + haunted hallways expa...", 252399, 2287],
    92: ["very bad lands survive the chaos!", 422692, 5071],
    93: ["very bad lands survive the chaos! brachio", 422692, 5073],
    94: ["viral - (+ the hive expansion)", 194690, 4356],
    95: ["woodtopia", 401976, 4686],
    96: ["you think you know me: a conversational card game", 287934, 4829],
    97: ["zombies!!! 20th anniversary edition", 2471, 4739],
}

df_str_2 = pd.DataFrame().from_dict(final_fill, orient="index")
df_str_2.columns = ["name", "bgg_id_fill", "min_titles_id"]
df_str_2["bgg_id_fill"] = df_str_2["bgg_id_fill"].astype(pd.Int64Dtype())

In [237]:
df_pax_fill4 = df_pax_fill3.merge(df_str_2[["min_titles_id", "bgg_id_fill"]], on=["min_titles_id"], how="left")
df_pax_fill4.loc[df_pax_fill4["bgg_id"].isna(), "bgg_id"] = df_pax_fill4["bgg_id_fill"]
df_pax_fill4 = df_pax_fill4.drop(columns=["bgg_id_fill"])
print(df_pax_fill4.shape)
display(df_pax_fill4.isna().sum())
df_pax_fill4.head()

(1831, 9)


name                 0
name_raw          1805
bgg_id              11
publisher            1
min_titles_id        0
titles_id_list       0
convention           0
year                 0
year_added           0
dtype: int64

,name,name_raw,bgg_id,publisher,min_titles_id,titles_id_list,convention,year,year_added
0,#artbox,NaN,280304,lavka games,5025,5025,unplugged,2024,2024
1,1001 islands,NaN,351972,ludonaute,4036,4036,unplugged,2024,2022
2,13 beavers,NaN,423850,format games,4996,4996,unplugged,2024,2024
3,18lilliput,NaN,250621,fox in the box,2808,2808,unplugged,2024,2019
4,1902 melies,NaN,393456,looping games,4539,4539,unplugged,2024,2023


In [238]:
query_time = int(time())
df_pax_fill4.to_csv(f"../../data/pax/pax_tt_games_{query_time}.csv", index=False,sep="|", escapechar="\\", quoting=csv.QUOTE_NONE)

In [242]:
[print(x) for x in df_pax_fill4.columns.tolist()]

name
name_raw
bgg_id
publisher
min_titles_id
titles_id_list
convention
year
year_added


[None, None, None, None, None, None, None, None, None]

In [42]:
# --- Normalize strings for better matching ---
def normalize(text):
    if pd.isna(text):
        return ""
    # Lowercase, remove accents, strip, simplify characters
    text = unicodedata.normalize('NFKD', text)
    text = "".join([c for c in text if not unicodedata.combining(c)])
    text = text.lower()
    text = text.replace("&", "and")
    text = re.sub(r"[^a-z0-9\s:]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Preprocess df_games with normalized name column
df_games["normalized_name"] = df_games["name"].apply(normalize)

# Optional: add alternate titles/aliases here
# e.g., df_games = df_games.append({"id": 999, "name": "arcs", "normalized_name": "arcs conflict and collapse in the reach"}, ignore_index=True)

# Match function using token_set_ratio and normalized text
def get_best_match(pax_name, games_df, threshold=80):
    norm_name = normalize(pax_name)
    match = process.extractOne(
        norm_name,
        games_df["normalized_name"],
        scorer=fuzz.token_set_ratio
    )
    if match and match[1] >= threshold:
        # Get full original record by index
        matched_row = games_df.iloc[match[2]]
        return matched_row["name"], matched_row["id"]
    return None, None

# Apply fuzzy matching
df_pax_null["matched_name"], df_pax_null["matched_id"] = zip(*df_pax_null["name"].apply(lambda x: get_best_match(x, df_games)))

In [50]:
# Step 1: Normalize strings
def normalize(text):
    if pd.isna(text):
        return ""
    text = unicodedata.normalize('NFKD', text)
    text = "".join([c for c in text if not unicodedata.combining(c)])
    text = text.lower()
    text = text.replace("&", "and")
    text = re.sub(r"[^a-z0-9\s:]", "", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

# Step 2: Custom matching function
def smart_match(pax_name, games_df, threshold=80):
    norm_pax = normalize(pax_name)
    tokens_pax = set(norm_pax.split())

    best_score = -1
    best_row = None

    for i, row in games_df.iterrows():
        norm_game = row['normalized_name']
        tokens_game = set(norm_game.split())

        base_score = fuzz.token_set_ratio(norm_pax, norm_game)

        # Boost if all query tokens are in candidate
        if tokens_pax.issubset(tokens_game):
            base_score += 10

        # Boost if query is substring of candidate
        if norm_pax in norm_game:
            base_score += 10

        if base_score > best_score:
            best_score = base_score
            best_row = row

    if best_score >= threshold:
        return best_row["name"], best_row["id"]
    return None, None

df_games["normalized_name"] = df_games["name"].apply(normalize)
df_pax_null["matched_name"], df_pax_null["matched_id"] = zip(*df_pax_null["name"].apply(lambda x: smart_match(x, df_games)))

KeyboardInterrupt: 

In [33]:
df_pax_null

,name,matched_name
0,#artbox,artbox
1,1902 melies,1902 méliès
2,6 nimmt!,11 nimmt!
3,9 peak,9peak
4,a game of folklore,None
5,adventure party,adventure park
6,aggretsuko,None
7,agueda city of umbrellas,None
8,alpha beasts attack!,None
9,arcs conflict & collapse in the reach,None


In [43]:
df_pax_null

,name,matched_name,matched_id
0,#artbox,artbox,280304.0
1,1902 melies,1902 méliès,393456.0
2,6 nimmt!,6%,37146.0
3,9 peak,peak,347865.0
4,a game of folklore,game of 45,37865.0
5,adventure party,don't be a dork: the ultimate party adventure ...,11530.0
6,aggretsuko,aggretsuko office rage card game,360088.0
7,agueda city of umbrellas,city,900.0
8,alpha beasts attack!,attack!,6752.0
9,arcs conflict & collapse in the reach,conflict,4325.0


In [45]:
df_pax_null["matched_name"].iloc[5]

"don't be a dork: the ultimate party adventure game!"

Some issues got resolved but there are still a few issues with the current matching.  Here are some examples 
- "adventure party" got matched to "don't be a dork: the ultimate party adventure game!" when the correct match was actually "adventure party: the role-playing party game".  
- "agueda city of umbrellas" got matched to "city" but should have been match to the normalized name "agueda: city of umbrellas"
- "9 peak" got matched to "peak" but should have been match to the normalized name "9peak"
- "ascension 10 year anniversary edition" got matched to "ascension" but should have been match to the normalized name "ascension: 10 year anniversary edition"
- "arcs conflict & collapse in the reach" now matched to "conflict" but should have matched to "arcs"
- "avatar: aang's destiny" got matched to "destiny" but should have matched to  "avatar: the last airbender – aang's destiny"
- "bananya" matched to "banana banana" but should have matched to "bananya: the card game"

There are a few issues with the current matching.  Here are the examples 
- "adventure party" got matched to "adventure park" when the correct match was actually "adventure party: the role-playing party game".  
- "agueda city of umbrellas" found no suitable matches but should have matched to "águeda: city of umbrellas"
- "aggretsuko" found no suitable matches but should have matched to "aggretsuko: work/rage balance" and 
- "arcs conflict & collapse in the reach" found no suitable matches but should have matched to "arcs"
- "avatar: aang's destiny" found no suitable matches but should have matched to  "avatar: the last airbender – aang's destiny"
- "bananya" matched to "bananas" but should have matched to "bananya: the card game"

In [36]:
df_games.loc[df_games["id"]==359871]

,id,name,rank,usersrated,is_expansion
116972,359871,arcs,123.0,10054,0


In [48]:
df_games.loc[df_games["normalized_name"].str.contains("ascension")]

,id,name,rank,usersrated,is_expansion,normalized_name
2641,3096,ascension at firepeak,23386.0,34,0,ascension at firepeak
8958,10403,ascension,19736.0,30,0,ascension
22289,25952,cetacean ascension moonbase 8,NaN,2,0,cetacean ascension moonbase 8
41136,67061,right ascension,NaN,2,0,right ascension
41895,69789,ascension: deckbuilding game,784.0,13857,0,ascension: deckbuilding game
45730,95064,ascension: return of the fallen,1200.0,3378,0,ascension: return of the fallen
48165,108784,ascension: storm of souls,1114.0,3673,0,ascension: storm of souls
51277,122294,ascension: immortal heroes,1910.0,1689,0,ascension: immortal heroes
55908,138233,ascension: rise of vigil,2047.0,1812,0,ascension: rise of vigil
57609,144864,ascension: darkness unleashed,2682.0,1044,0,ascension: darkness unleashed


In [39]:
df_pax_null.isna().sum()

name             0
matched_name    85
dtype: int64

In [60]:
df_parq = pd.read_parquet("../../data/crawler/boardgame_data_1750469667.parquet",columns=["id"])
df_parq.shape

(164225, 1)

In [112]:
boardgame_ranks_raw = pd.read_csv("../../data/crawler/boardgame_ranks_20250526.csv", sep="|", escapechar="\\", quoting=csv.QUOTE_NONE)
boardgame_ranks = boardgame_ranks_raw.copy()
boardgame_ranks["name"] = boardgame_ranks["name"].str.replace("[“”]", '"', regex=True)
boardgame_ranks.shape

(165498, 17)

In [117]:
boardgame_ranks.to_csv(
    "../../data/crawler/boardgame_ranks_20250527.csv",
    index=False,
    sep="|",
    escapechar="\\",
    quoting=csv.QUOTE_NONE,
)

In [120]:
boardgame_data =  pd.read_parquet("../../data/crawler/boardgame_data_1750471875.parquet")

In [69]:
boardgame_ids = boardgame_ranks["id"].tolist()

In [71]:
boardgame_master_dict = {
            str(x["id"]): x for x in boardgame_data.to_dict(orient="records")
        }

In [80]:
boardgame_ranks.shape

(164257, 17)

In [ ]:
boardgame_ranks["name"].str.replace("[“]", regex=True)

In [110]:
chr(8220)

'“'

In [118]:
boardgame_data.shape

(164225, 37)

In [119]:
165498-164225

1273

In [4]:
x_h = 89.6
x_w = 55.2
b_h = 115
b_w = 75

x_h/b_h

0.7791304347826087

In [20]:
# math.cos(30*math.pi/180)*100
# 106.74/math.cos(30*math.pi/180)
# (100**2+6**2)**0.5
# x_h/b_h*137
20.5*25.4/3

173.56666666666663